In [ ]:
!pip install dataretrieval -q

In [ ]:
import datetime
import json
import requests

import ee
import geemap

import dataretrieval.nwis as nwis

In [ ]:
#@title Enter Cloud Project to use with Earth Engine
# Please fill in these values.
PROJECT = ""  # @param {type:"string"}

PROJECT = None if PROJECT == "" else PROJECT

# Quick input validations.
if not PROJECT:
    raise ValueError("⚠️ A Google Cloud project was not provided. Please provide a Google Cloud project ID to authenticate EE against.")

In [ ]:
# initialize Earth Engine client
ee.Initialize(project=PROJECT)

In [ ]:
# @title Enter start and end dates for processing
start_date = '2020-01-01' # @param {type:"date"}
end_date = "" # @param {type:"date"}

if end_date == "" or end_date is None:
    end_date = datetime.datetime.now().strftime('%Y-%m-%d')

In [ ]:
# specify the USGS site code for which we want data.
# this site corresponds with the Meduxnekeag River in Maine
# https://waterdata.usgs.gov/nwis/inventory?agency_code=USGS&site_no=01018035
site = '01018035'

gauge_lon_lat = (-67.805, 46.176)

In [ ]:
# get the daily values for the last 40 years from the site with the parameter code 00060 (discharge)
gauge = nwis.get_record(sites=site, service='dv', start=start_date, end=end_date,parameterCd='00060')
gauge.mask(gauge['00060_Mean'] < 0, inplace=True)
gauge.index = gauge.index.tz_localize(None)

In [ ]:
# load in the HUC8 watersheds from Earth Engine
huc08_basins = ee.FeatureCollection("USGS/WBD/2017/HUC08")

In [ ]:
# get the gauge as a point geometry in Earth Engine
ee_gauge = ee.Geometry.Point(*gauge_lon_lat)

# filter the watersheds to the basin the gauge is in
basin = ee.Feature(
    huc08_basins
    .filterBounds(ee_gauge)
    .first()
)

In [ ]:
# visualize the basin and gauge on a map
m = geemap.Map()
m.add_layer(basin, {}, 'Basin')
m.add_layer(ee_gauge, {'color':'yellow'}, 'Gauge')
m.center_object(ee_gauge, 10)
m

In [ ]:
# get the area of the basin
# this is used to normalize discharge
area_sqm = float(basin.geometry().area().getInfo())

In [ ]:
# normalize discharge from cubic feet per second to mm per day
gauge['mm/day'] = 28316846.592 * gauge['00060_Mean'] * 86400 / (area_sqm * 1e6)

In [ ]:
# plot the gauge data
gauge['mm/day'].plot();

In [ ]:
# specify the names of the features/bands we use
# these must correspond to the dynamic_inputs used to train the model
in_features = ['prcp','srad','tmax','tmin','vp']

In [ ]:
# load in the daymet collection and filter to dates
# select out the features to input into model
daymet_collection = (
    ee.ImageCollection('NASA/ORNL/DAYMET_V4')
    .filterDate(start_date, end_date)
    .select(in_features)
)

> **NOTE:** If you used another meteorological forcing to train the network then make sure you update the image collection to the same one used to train the model

In [ ]:
#@title Time series function
def get_timeseries(forcings, basin):
    """Calcualte a time series of forcings for a given basin.

       args:
           basin (ee.Feature): Feature of the basin to calculate the time series for

       returns:
           pd.DataFrame: table of time series of forcings
    """

    def calc_ts(t):
        # determine start and end time for the reduction
        t1 = ee.Date(t)
        t2 = t1.advance(1, 'day')

        # get the first image within time range
        img = forcings.filterDate(t1, t2).first()

        # apply the reduction for the areal avg precip
        stat = img.reduceRegion(
            geometry=basin.geometry(maxError=1e3),
            reducer=ee.Reducer.mean(),
            scale=img.projection().nominalScale(),
            tileScale=2
        )

        # update the reduction result with time info
        stat = stat.combine({
            'date': t1.format('YYYY-MM-dd HH:mm:ss'),
            'system:time_start': t1.millis()
        })
        return ee.Feature(None, stat)  # return as a feature with no geometry

    # get the years for a time series
    datetimes = forcings.aggregate_array('system:time_start')

    # map over all of the forecast times and convert to FeatureCollection
    ts = ee.FeatureCollection(datetimes.map(calc_ts))

    # request that the table be returned as pd.DataFrame
    return ee.data.computeFeatures({
        'expression': ts,
        'fileFormat': 'PANDAS_DATAFRAME'
    })

In [ ]:
# request a time series of the meteorological inputs for the basin
daymet_df = get_timeseries(daymet_collection, basin)
daymet_df.index = pd.to_datetime(daymet_df.date)

Depending on the size of basin this can timeout. If you would like to run this for large or many basins then it is better to [export the table](https://developers.google.com/earth-engine/guides/exporting_tables) rather than to read in a DataFrame

In [ ]:
# display resulting dataframe
daymet_df.head()

In [ ]:
# plot each feature
daymet_df[in_features].plot(subplots=True, figsize=(7, 7));

In [ ]:
#@title Enter endpoint to use for predictions

prediction_endpoint = 'https://us-central1-ciroh-water-demo.cloudfunctions.net/nh-inference' # @param {type:"string"}


In [ ]:
# convert the dataframe values into a list to send for prediction
in_data = daymet_df[in_features].values.tolist()

In [ ]:
# run prediction using the endpoint
r = requests.post(prediction_endpoint, json={'forcings': in_data})

In [ ]:
# check the response and if okay then get the result as an array
if r.status_code == 200:
    q_out = np.array(r.json()['y_pred'])
else:
  print(r.json())

In [ ]:
# convert the array to a time series
q_df = pd.Series(np.squeeze(q_out), index=daymet_df.index[365:])

In [ ]:
# plot the results with the observed data
ax = q_df.plot(label='Prediction')
gauge['mm/day'].plot(ax=ax,label='Observed', color='k')
ax.legend();